1. Run "pip install kaggle" 

2. Move "kaggle.json" to following Path /Users/linh/.kaggle

Importing relevent libraries

In [8]:
import pandas as pd 
from cleaning import *  # Import all functions from cleaning
import kaggle

In [ ]:
# Initiliaze API

api = KaggleApi()
api.authenticate()
api.model_list_cli()

Next Page Token = [...]

In [5]:
# Saving CSV into DF
india_df = pd.read_csv("./Data/Cardiovascular_Disease_Dataset.csv")
india_df

In [10]:
# Working through API and Call new Data per api 
"https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset"

'https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset'

Clean Data

In [11]:
# Checking and konsolodate the columns

In [13]:
# Concantenate the DataFrames

Preparing the Data Analysis

In [14]:
# Defining Hypothesis and relevant Data

Visualizing the Data

In [15]:
# Importing relevant Liberaries

In [16]:
# Defining relevant Plots